In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jan  4 14:16:57 2020

@author: Roberto Villalobos

 Sub-hourly climatology script
 Will calculate statistics at various durations in a similar fashion to TBR_desc_stats_auto.R
 Output should preserve same format as R script
 Resulting data will be visualised in R using desc_stats_explorer.R
 
 
 Next speed-up: save data from all durations, for one station, and write once instead of writing at each duration
"""

# -----------------------------------------------------------------------------
# Packages
import os
#os.chdir('D:/PhD/13. Scripts/phd-python-code')
os.chdir('C:/PhD/13. Scripts/phd-python-code/Intense_QC/')
##import intense.intense as ex
import intense_Roberto_03 as ex
import glob
import os.path
import datetime
import numpy as np
import pandas as pd
import scipy.stats as stats
#import UKQC.intense_CW as ex
from joblib import Parallel, delayed


import xarray as xr
import numpy.ma as ma


def P_missing(series):
    return(max(100*series.isna().sum() / series.shape[0], 100*(1-series.shape[0]/(24*365))))
    
def P_missing_NL(series):
    return(max(100*series.isna().sum() / series.shape[0], 100*(1-series.shape[0]/(24*30))))
    
    
# ETCCDI utility function 1
def prep_etccdi_variable(input_path, index_name, aggregation, data_source):
    ds = xr.open_dataset(input_path)

    # Omit final year (2010) of HADEX2 - suspiciously large CDD for Malaysia
    if data_source == 'HADEX2':
        ds = ds.sel(time=slice(datetime.datetime(1951, 1, 1, 0),
                               datetime.datetime(2009, 12, 31, 23)))

    # Calculate maximum rainfall value over whole period
    vals = ds[index_name].values
    if index_name in ['CWD', 'CDD']:
        vals = ds[index_name].values.astype('timedelta64[s]')
        vals = vals.astype('float32') / 86400.0
        vals[vals < 0.0] = np.nan
    vals = ma.masked_invalid(vals)
    if aggregation == 'max':
        data = ma.max(vals, axis=0)
    if aggregation == 'mean':
        data = ma.mean(vals, axis=0)

    # Convert back from to a xarray DataArray for easy plotting
    # - masked array seems to be interpreted as np array (i.e. nans are present
    # in the xarray DataArray
    data2 = xr.DataArray(data, coords={'Latitude': ds['lat'].values,
                                       'Longitude': ds['lon'].values}, dims=('Latitude', 'Longitude'),
                         name=index_name)

    ds.close()

    return data2
# ETCCDI utility function 2
def get_etccdi_value(etccdi_data, index_name, lon, lat):
    lon = float(lon)
    lat = float(lat)

    # Check gauge longitude and convert to -180 - 180 range if necessary
    if lon > 180.0:
        lon = lon - 360.0

    # Array location indices for closest cell centre to gauge location
    location_indices = {'GHCNDEX': {}, 'HADEX2': {}}
    for data_source in location_indices.keys():
        location_indices[data_source]['lon'] = (np.argmin(
            np.abs(etccdi_data[data_source][index_name]['Longitude'].values - lon)))
        location_indices[data_source]['lat'] = (np.argmin(
            np.abs(etccdi_data[data_source][index_name]['Latitude'].values - lat)))

    # Maximum of ETCCDI index values from GHCNDEX and HADEX2 for cell
    etccdi_index_values = {}
    for data_source in location_indices.keys():
        yi = location_indices[data_source]['lat']
        xi = location_indices[data_source]['lon']
        etccdi_index_values[data_source] = etccdi_data[data_source][index_name].values[yi, xi]
    etccdi_vals = np.asarray(list(etccdi_index_values.values()))
    if np.any(np.isfinite(etccdi_vals)):
        max_index = np.max(etccdi_vals[np.isfinite(etccdi_vals)])
    else:
        max_index = np.nan

    # For cases where no value for the cell, look in 3x3 window and take the maximum
    if np.isnan(max_index):
        etccdi_index_window = {}
        for data_source in location_indices.keys():
            yi = location_indices[data_source]['lat']
            xi = location_indices[data_source]['lon']
            window = etccdi_data[data_source][index_name].values[yi - 1:yi + 2, xi - 1:xi + 2]
            if np.any(np.isfinite(window)):
                etccdi_index_window[data_source] = np.max(window[np.isfinite(window)])
            else:
                etccdi_index_window[data_source] = np.nan

        window_vals = np.asarray(list(etccdi_index_window.values()))
        if np.any(np.isfinite(window_vals)):
            max_index_window = np.max(window_vals[np.isfinite(window_vals)])
        else:
            max_index_window = np.nan

    else:
        max_index_window = np.nan

    return max_index, max_index_window

# Prepare ETCCDI variables
def read_etccdi_data(etccdi_data_folder):
    etccdi_data = {"GHCNDEX": {}, "HADEX2": {}}
    etccdi_indices = ['CWD', 'CDD', 'R99p', 'PRCPTOT', 'SDII', 'Rx1day']
    periods = {"GHCNDEX": '1951-2018', "HADEX2": '1951-2010'}
    aggregations = {}
    for index in etccdi_indices:
        aggregations[index] = 'max'
    aggregations['SDII'] = 'mean'
    for data_source in etccdi_data.keys():
        for index in etccdi_indices:
            etccdi_data_path = (etccdi_data_folder + '/RawData_' + data_source +
                                '_' + index + '_' + periods[data_source] +
                                '_ANN_from-90to90_from-180to180.nc')
            etccdi_data[data_source][index] = prep_etccdi_variable(etccdi_data_path,
                                                                   index, aggregations[index], data_source)
    return etccdi_data
  

# Calculate critical interarrival time       
def crit_dur(file, metadir, etccdi_data):
    # Read station data
    try:
        data = pd.read_csv(file)
        
        # Get datetime index
        data.index = pd.DatetimeIndex(data['ob_time'])
        # Get metadata in file
        station_id = data['id'][1]
        station_name = data['src_id'][1]
    except:
        print('Could not read data for '+file)
    
    # Additional metadata
    try:
        metadata = ex.readIntense(metadir+station_id+".txt", only_metadata = False)
        lon = metadata.longitude
        lat = metadata.latitude
    except:
        lon = -999
        lat = -999
        print('Could not read metadata for '+file)
        
    # Drop un-needed columns
    data = pd.DataFrame(data['accum'])
    # Calculate time difference vector to identify if data is 15-min or other type
    tdifs = data.index.to_series().diff()/np.timedelta64(1,'s')
    tdifs = tdifs.resample('M').apply(lambda x: stats.mode(x)[0]) 
    
    # If at least one month has 15-min data, consider the entire record as 15-min
    if tdifs[tdifs == 900].shape[0] >= 1:
        tdif = 900
    else:
        tdif = 60

    # -------------------------------------------------------------------------
    # Calculate percentage missing values 
    # Converting float to strings forces count to include all values and not ignore nans
    # Need to re-work pdifs to account for very few instances where SHQC-removed events 
    # add lots of nans in terms of observations but not in terms of time (see Andover case)

    # Percentage missing, per month:
    pdifsm = 100*((data.astype('str').resample("M").count() - data.resample("M").count())/data.astype('str').resample("M").count())
    pdifsm.rename(columns={"accum":"p_missing"}, inplace = True)
    
    # Percentage missing, according to hourly data, this is a superior method:
    # Mixed mode because of manually deleted years in SubH data not found in H data
    pdifshm = metadata.data.resample('M').apply(P_missing_NL)
    
    pdifshm = pd.concat([pdifsm, pdifshm], axis = 1)
    pdifshm[0] = np.where((pdifshm[0] == 100*(1-28/30))|(pdifshm[0] == 100*(1-29/30)),0,pdifshm[0])
    
    # Instead of years over which to iterate, we want a corrected number of years, n
    # with useable data for out k largest
    
    # Remove months with missing data
    months = pdifshm[pdifshm < 15].dropna().shape[0]
    good_months = pdifshm[pdifshm < 15].dropna().index.strftime("%Y%m")
    
    # Number of years with less than 15% missing data
    n = int(months/12)

    
    output= pd.DataFrame(columns = ["Station_id","Station_name","Lon","Lat",
                                    "Critical_interarrival_time"])
   
    # If there is at least one complete year:
    if n >= 1:
    
        data = data.dropna()
        dur = 15
        # We first resample to 15-minutes as this is the smallest common duration
        rolling = data['accum'].resample(str(dur)+'min', label = 'right').sum()
        # Next we create a time series where we remove all the zero values
        wet_only = np.where(rolling == 0, np.nan, rolling)
        rolling = pd.DataFrame(rolling)
        rolling['non_zero'] = wet_only
        # Now we keep only this column and drop nas
        rolling = rolling['non_zero'].copy()
        rolling = rolling.dropna()
        # We can now calculate the non-rainy period durations
        bi = rolling.index.to_series().diff()/np.timedelta64(1,'s')
        
        # Remove missing data gaps
        bi = pd.DataFrame(bi)
        bi['ym'] = bi.index.strftime("%Y%m")
        #bi = bi.loc[bi['ym'].isin(good_months)] # This line removes missing data, does not change results by much
        bi = bi['ob_time']
        # Every difference of 900s is due to consecutive wet periods
        # If we remove them we are left with the duration of non-rainy periods
        bi = bi.loc[bi != 900] 
        # Remove also intervals over 28 days
        #bi = bi.loc[bi < 28*24*60*60]
        # Use a longer 3 month interval to test sensistivity
        #bi = bi.loc[bi < 3*30*24*60*60]
        # Use a 6-month gap
        #bi = bi.loc[bi < 6*30*24*60*60]
        # After checking individual gauges that show changes: all observed changes due to existance of 
        # real missing data, considering a monthly duration is used to filter missing data,
        # A suggested value of 31 days max gap is allowed. 
        #bi = bi.loc[bi < 31*24*60*60]
        
                # Use CDD index as solution to define maximum allowable dry period
        # By definition it is the ideal solution

        longest_dry_period, longest_dry_period_filled = get_etccdi_value(etccdi_data, 'CDD', lon, lat)
        if np.isnan(longest_dry_period):
            if np.isnan(longest_dry_period_filled):
                # Default to 30 days if needed
                bi = bi.loc[bi < 30*24*60*60]
            else:
                bi = bi.loc[bi< longest_dry_period_filled*24*60*60]
        else:
            bi = bi.loc[bi< longest_dry_period*24*60*60]

        
        hist_bi=np.histogram(bi, bins = np.linspace(0,np.nanmax(bi),int(np.nanmax(bi)/900)))
        
        # N0 is the total number of non-rainy periods
        h = hist_bi[0]
        s1_0 = (1/len(bi))*np.nansum(bi)
        s2_0 = (1/len(bi))*np.nansum(bi**2)
        nk = []
        nk.append(len(bi))
        s1k = []
        s1k.append(s1_0)
        s2k = []
        s2k.append(s2_0)
        cv = []
        cv.append( (1/s1k[0])*(((s2k[0]-s1k[0]**2)*(nk[0]/(nk[0]-1)))**0.5 ))
        # Calculate N_k = N_k-1 - h_k
        crit_dur = np.nan
        for i in range(1,len(h)):
            ni = nk[i-1] - h[i]
            nk.append(ni)
            s1_i = (1/ni)*(nk[i-1]*s1k[i-1]-h[i]*(i)) 
            s1k.append(s1_i)
            s2_i = (1/ni)*(nk[i-1]*s2k[i-1]-h[i]*(i)**2) 
            s2k.append(s2_i)
            cv.append( (1/s1k[i])*(((s2k[i]-s1k[i]**2)*(nk[i]/(nk[i]-1)))**0.5 ))
            if cv[i] <= 1:
                crit_dur = (i-1)*900/(60**2)
                break
            
        output = output.append({"Station_id": station_id,
                                "Station_name":station_name,
                                "Lon": lon,"Lat":lat,
                                "Critical_interarrival_time":crit_dur}, 
                                                    ignore_index = True)
        return output
    else:
        print('Station '+station_id+' does not have complete years.')



#%%
        
etccdi_data =  read_etccdi_data(r"C:\Rainfall data\ETCCDI")        
        
# Just ams after adding overlap code, takes about 30 mins to run... more like 3 mins in the new laptop!
in_dir = "C:/Ultimate_QC/FINAL_HQC_FM_SHQC_M_SHData"
metadir = "C:/Ultimate_QC/QCd_Data/FL13UK/"

files = glob.glob(os.path.join(in_dir,"*.txt"))
#files.sort(reverse = True)
#file = files[2]
logdir = "C:/Ultimate_QC/Climatology_RX_cor/Joined2"

logdir2 = logdir+"/interarrival_thresholds_CDD_noMissing.txt"

out = Parallel(n_jobs = 14, verbose = 9)(delayed(crit_dur)(file, metadir, etccdi_data) for file in files)

#out = [x for x in out if x != None]

ams_out  = pd.concat([item for item in out])

ams_out.to_csv(logdir2, index = False)
#crit_dur(files[5], metadir)
#file = in_dir+"/EA362710501.txt" # large gap of real missing data
#file = in_dir+"/SEPA525510.txt" # real, 35 day gap
# Test effect of 28 to 31 day change
#file = in_dir+"/EA999998.txt"


#np.nanmean(ams_out["Critical_interarrival_time"])
#%%
# Find 5-day max, Honister tel

file = in_dir+"/EA592463.txt"

# Read station data
try:
    data = pd.read_csv(file)
    
    # Get datetime index
    data.index = pd.DatetimeIndex(data['ob_time'])
    # Get metadata in file
    station_id = data['id'][1]
    station_name = data['src_id'][1]
except:
    print('Could not read data for '+file)

# Additional metadata
try:
    metadata = ex.readIntense(metadir+station_id+".txt", only_metadata = False)
    lon = metadata.longitude
    lat = metadata.latitude
except:
    lon = -999
    lat = -999
    print('Could not read metadata for '+file)
    
# Drop un-needed columns
data = pd.DataFrame(data['accum'])
rolling5d = data['accum'].rolling('7200 min').sum()
rolling5d.loc[rolling5d == max(rolling5d)].index

#%% Updates

etccdi_data =  read_etccdi_data(r"C:\Rainfall data\ETCCDI")        
        
# Just ams after adding overlap code, takes about 30 mins to run... more like 3 mins in the new laptop!
metadir = r"D:\Rainfall_storage\2020updates\QC\QCd_Data\updates/"
in_dir = r"D:\Rainfall_storage\2020updates\QC\QCd_Data\updates_SH_FR_SHQCd"
 
files = glob.glob(os.path.join(in_dir,"*.txt"))
#files.sort(reverse = True)
#file = files[2]
logdir = "D:/Rainfall_storage/2020updates/QC/Summary"

logdir2 = logdir+"/interarrival_thresholds_CDD_noMissing.txt"

out = Parallel(n_jobs = 14, verbose = 9)(delayed(crit_dur)(file, metadir, etccdi_data) for file in files)

#out = [x for x in out if x != None]

ams_out  = pd.concat([item for item in out])

ams_out.to_csv(logdir2, index = False)
